In [2]:
import argparse
import os
import random
import sys
import time

import numpy as np

import ite
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


In [6]:
"""
Simple 5 layer perceptron autoencoder. 
"""


class Net(nn.Module):
    def __init__(self, input_size=45, hidden_size=45, bottleneck_size=20):
        super(Net, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bottleneck_size = bottleneck_size

        # ENCODER
        self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
        self.fc2 = torch.nn.Linear(self.hidden_size, self.hidden_size)
        self.fc3 = torch.nn.Linear(self.hidden_size, self.bottleneck_size)

        # DECODER
        self.fc4 = torch.nn.Linear(self.bottleneck_size, self.hidden_size)
        self.fc5 = torch.nn.Linear(self.hidden_size, self.hidden_size)
        self.fc6 = torch.nn.Linear(self.hidden_size, self.input_size)

    def forward(self, x):
        enc = F.leaky_relu(self.fc1(x), 0.1)
        enc = F.leaky_relu(self.fc2(enc), 0.1)
        enc = F.leaky_relu(self.fc3(enc), 0.1)

        dec = F.leaky_relu(self.fc4(enc), 0.1)
        dec = F.leaky_relu(self.fc5(dec), 0.1)
        dec = F.leaky_relu(self.fc6(dec), 0.1)
        return enc, dec


# parser = argparse.ArgumentParser()
# parser.add_argument(
#     "--ppg_path",
#     help="Path of Normalized PPG numpy array with each element representing the PPG signal",
#     default="../data/flatten_ppg.npy",
# )

# parser.add_argument(
#     "-label_path",
#     help="Path of PPG numpy array with each element representing the PPG signal",
#     default="../data/flatten_sbp.npy",
# )

# parser.add_argument("-ite_path", help="Path to the ite-repo", default=None)
# parser.add_argument(
#     "-emb_dim", help="size of the bottleneck layer embeddings", default=20
# )

# parser.add_argument(
#     "-neighbors",
#     help="neighbor parameter for the estimation of mutual information. Higher values reduce the variance but might introduce a bias",
#     default=100,
# )

# parser.add_argument("-device", help="GPU or CPU", default="cuda:0")


# args = parser.parse_args()

# # Loading the ite-library for mutual information estimation. Check https://bitbucket.org/szzoli/ite-in-python/
# sys.path.insert(
#     1,
#     "/home/t-surilmehta/home/szzoli-ite-in-python-44a8f15e2dc9/szzoli-ite-in-python-44a8f15e2dc9",
# )

# Loading data and parameters
ppg = test_4
label = test_sbp
emb_dim =20
# device = args.device
input_size = ppg.shape[1]

# Initializing model and optimizers
model = Net(input_size, input_size, emb_dim).to(device)
optim = torch.optim.Adam(model.parameters(), lr=1e-5)

# Conversion to tensors
X, Y = torch.FloatTensor(ppg), torch.FloatTensor(label)
train_dataset = TensorDataset(X, Y)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)

epochs = 10000
from tqdm import trange
# Training+Validation loop
for epoch in trange(epochs + 1):
    optim.zero_grad()
    train_loss, test_loss = 0, 0
    model.train()
    for inp, label in train_loader:
        inp, label = inp.to(device), label.to(device)
        with torch.set_grad_enabled(True):
            enc, out = model(inp)
            loss = F.mse_loss(out, inp)
            loss.backward()
            optim.step()
        train_loss += loss.item()

    model.eval()
    y, encodings = [], []
    for inp, label in train_loader:
        inp, label = inp.to(device), label.to(device)
        with torch.set_grad_enabled(False):
            enc, out = model(inp)
            loss = F.mse_loss(out, inp)
            encodings.extend(enc.cpu().detach().numpy())
            y.extend(label.cpu().detach().numpy())
        test_loss += loss.item()

    train_loss, test_loss = (
        round(train_loss / len(train_loader), 3),
        round(test_loss / len(train_loader), 3),
    )

    # Logging the loss values and estimating mutual information.
    if epoch % 100 == 0:
        print(f"Epoch:{epoch} Train_Loss:{train_loss} Test_Loss:{test_loss}")

    if train_loss < 0.1 or test_loss < 0.1 and epoch % 5 == 0:
        print(f"Epoch:{epoch} Train_Loss:{train_loss} Test_Loss:{test_loss}")
        encodings, y = np.asarray(encodings), np.asarray(y).reshape(-1, 1)
        co = ite.cost.MIShannon_DKL(
            mult=True, kl_co_name="BDKL_KnnK", kl_co_pars={"k": 100}
        )
        # emb_dim = 20
        print(encodings.shape, y.shape)
        val = co.estimation(np.hstack((encodings, y)), [emb_dim, 1])
        print(f"MI value:{val}")
        co_HY = ite.cost.BHShannon_KnnK(mult=True, k=100)
        H_Y = co_HY.estimation(y.reshape(-1,1))
        print(f"Entropy of target (H_Y): {H_Y}")
        info_fraction = val / H_Y
        print(f"Info-Fraction: {info_fraction}")

NameError: name 'device' is not defined

## load  input data

In [3]:
test_2 = np.load("F:\\minowa\\BloodPressureEstimation\\data\\processed\\BP_npy\\PulseDB\\test_2.npy")[:,1]
test_sbp_2 = np.load("F:\\minowa\\BloodPressureEstimation\\data\\processed\\BP_npy\\PulseDB\\test_sbp_2.npy")
test_dbp = np.load("F:\\minowa\\BloodPressureEstimation\\data\\processed\\BP_npy\\PulseDB\\test_dbp.npy")
test_4 = np.load("F:\\minowa\\BloodPressureEstimation\\data\\processed\\BP_npy\\PulseDB\\test_4.npy").squeeze()
test_sbp = np.load("F:\\minowa\\BloodPressureEstimation\\data\\processed\\BP_npy\\PulseDB\\test_sbp.npy")
test_2.shape,test_sbp_2.shape,test_dbp.shape,test_4.shape,test_sbp.shape

((38110, 1250), (38110,), (111600,), (111600, 1250), (111600,))

In [4]:
test_sbp_2labels = np.load("F:\\minowa\\BloodPressureEstimation\\data\\processed\\BP_npy\\PulseDB\\test_sbp_2labels.npy")
test_sbp_4labels = np.load("F:\\minowa\\BloodPressureEstimation\\data\\processed\\BP_npy\\PulseDB\\test_sbp_4labels.npy")

In [5]:
test_sbp_4labels[:100]

array([1, 3, 2, 2, 2, 3, 3, 2, 2, 2, 2, 3, 2, 3, 3, 2, 3, 3, 3, 3, 1, 2,
       0, 1, 3, 1, 1, 3, 3, 1, 1, 1, 3, 3, 3, 3, 2, 3, 1, 2, 3, 2, 2, 1,
       3, 1, 3, 1, 3, 2, 3, 3, 3, 1, 2, 2, 2, 3, 3, 1, 2, 3, 2, 1, 3, 1,
       3, 3, 1, 3, 3, 1, 2, 3, 2, 1, 3, 2, 2, 3, 3, 2, 3, 1, 2, 2, 2, 3,
       2, 2, 3, 2, 1, 2, 1, 1, 2, 3, 2, 3])

In [16]:
test_2[1,0,:100]

array([ 83.05106629,  81.75012088,  80.38314718,  79.07587822,
        77.08756172,  75.06585372,  73.38349937,  71.82009373,
        70.74982242,  69.65830052,  68.60469233,  68.44008512,
        68.22482364,  67.37617673,  68.01488514,  68.7227993 ,
        68.41480079,  68.9993008 ,  68.99467477,  68.44390371,
        68.41623601,  68.39353164,  68.45823385,  67.83598993,
        67.18228907,  67.2565631 ,  67.20980974,  67.07671381,
        67.56650494,  69.59775776,  73.11195278,  77.97092736,
        84.60065154,  92.76061522, 101.99761091, 111.38650746,
       120.59963303, 128.9817073 , 135.31293323, 139.8897224 ,
       142.64108908, 142.95101796, 140.75307725, 136.91826539,
       132.10592029, 125.94105043, 119.65114131, 114.07136866,
       108.0501885 , 102.65635919,  98.40514212,  94.64634219,
        92.44722388,  90.79880825,  89.25317731,  88.70422841,
        88.22872354,  87.46775458,  86.11075563,  84.59287798,
        83.45771371,  81.74131707,  80.0203382 ,  78.50

In [7]:
from resnet1d import ResNet1D
config = {
    "network":{
        "in_channels":1,
        "base_filters":64,
        "kernel_size":3,
        "stride":1,
        "groups":1,
        "n_block":5,
        "n_classes":2,
    },
    "learning_rate": 1e-3,
    "batch_size": 32,
    "epochs":100,
    "log_interval":100,
    "input_shape":[1,1250],
    "output_path": "..\\outputs\\resnet\\0228",
    "ppg":10    
          }
model = ResNet1D(**config["network"])

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
model.load_state_dict(torch.load('..\\outputs\\resnet\\0228\\best.pth'))
model.to(device)
model.eval()

C:\Users\bsa\AppData\Local\Temp\ipykernel_39216\1154037463.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('..\\outputs\\resnet\\0228\\b

ResNet1D(
  (first_block_conv): MyConv1dPadSame(
    (conv): Conv1d(1, 64, kernel_size=(3,), stride=(1,))
  )
  (first_block_bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (first_block_relu): ReLU()
  (basicblock_list): ModuleList(
    (0-3): 4 x BasicBlock(
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU()
      (do1): Dropout(p=0.5, inplace=False)
      (conv1): MyConv1dPadSame(
        (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
      )
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU()
      (do2): Dropout(p=0.5, inplace=False)
      (conv2): MyConv1dPadSame(
        (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
      )
      (max_pool): MyMaxPool1dPadSame(
        (max_pool): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (4): BasicBlock(
      (bn1): 

In [10]:
outputs = {}

def hook_fn(module, input, output):
    outputs['features'] = output
model.final_relu.register_forward_hook(hook_fn)



In [11]:
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

scaler = StandardScaler()
# encodings_scaled = scaler.fit_transform(encodings)

pca = PCA(n_components=20)
# encodings_reduced = pca.fit_transform(encodings_scaled)

In [ ]:
X_ = torch.from_numpy(test_2).float().unsqueeze(1).to(device)
y_ = torch.from_numpy(test_sbp_2).float().to(device)
dataset = TensorDataset(X_,y_)
loader = DataLoader(dataset, batch_size=32)
encodings = []
y_out = []
for X,y in tqdm(loader):
    _ = model(X)
    out = outputs['features'].clone()
    # print(out.shape)
    out = out.mean(-1)
    out = out.flatten(start_dim=1).cpu().detach().numpy()

    
    # print(out.shape)
    encodings.append(out)
    y_out.extend(y.cpu().detach().numpy())
encodings= np.concatenate(encodings)

scaler = StandardScaler()
encodings_scaled = scaler.fit_transform(encodings)
pca = PCA(n_components=20)
encodings_reduced = pca.fit_transform(encodings_scaled)
y = np.array(y_out).reshape(-1, 1)
co = ite.cost.MIShannon_DKL(
    mult=True, kl_co_name="BDKL_KnnK", kl_co_pars={"k": 10}
)


emb_dim =20
stack = np.hstack((encodings_reduced, y))
print(stack.shape)
val = co.estimation(stack, [emb_dim, 1])
print(f"MI value:{val}")

# Y のみでエントロピー推定
co_HY = ite.cost.BHShannon_KnnK(mult=True, k=10)
H_Y = co_HY.estimation(y.reshape(-1,1))
print(f"Entropy of target (H_Y): {H_Y}")
info_fraction = val / H_Y
print(f"Info-Fraction: {info_fraction}")

100%|██████████| 1191/1191 [00:05<00:00, 218.76it/s]


(38110, 21)
MI value:10.338875400195587
Entropy of target (H_Y): 3.979911211307363
Info-Fraction: 2.597765339794946


In [63]:
co_entropy = ite.cost.HShannon_DKL(mult=False, kl_co_name="BDKL_KnnK", kl_co_pars={"k": 20})
H_Y = co_entropy.estimation(y)
H_Y

AttributeError: module 'ite.cost' has no attribute 'HShannon_DKL'

In [ ]:
# for epochs
fo


# features


In [14]:
import pandas as pd

In [ ]:
features = pd.read_csv('F:\\minowa\\BloodPressureEstimation\\data\\results\\ppg_features_pulsedb_test_min180.csv',index_col=0)

# target = pd.read_csv('F:\\minowa\\BloodPressureEstimation\\data\\results\\target.csv',index_col=0)
# features.shape,target.shape

((87507, 102), (6974, 7))

In [ ]:
# sort by signal_index
# features = features.sort_values("signal_index")
# target = target.loc[features.index]

In [17]:
target.head()

,err_DBP,err_SBP,abs_err_DBP,abs_err_SBP,target_SBP,target_DBP,signal_index
0,-6.893147,-9.505203,6.893147,9.505203,1,1,0
1,-5.536797,-5.235329,5.536797,5.235329,1,1,1
2,-18.116745,6.519360,18.116745,6.519360,1,2,2
3,-36.884052,-19.033958,36.884052,19.033958,2,2,3
4,-21.217396,-3.074295,21.217396,3.074295,0,2,4


In [50]:
# remove signal_index
y = features['sbp'].values.reshape(-1,1)
X = features.drop(columns=["signal_index","subject","sbp","dbp"])
X.shape,y.shape

((87507, 98), (87507, 1))

In [41]:
features.columns

Index(['Tc', 'Ts', 'Td', 'Tsteepest', 'Steepest', 'TNegSteepest',
       'NegSteepest', 'TdiaRise', 'DiaRise', 'SteepDiaRise', 'TSystoDiaRise',
       'TdiaToEnd', 'Ratio', 'Ts_norm', 'Td_norm', 'Tsteepest_norm',
       'TNegSteepest_norm', 'TdiaRise_norm', 'TSystoDiaRise_norm',
       'TdiaToEnd_norm', 'SW25', 'SW25_norm', 'DW25', 'DW25_norm', 'SWaddDW25',
       'SWaddDW25_norm', 'DWdivSW25', 'SW50', 'SW50_norm', 'DW50', 'DW50_norm',
       'SWaddDW50', 'SWaddDW50_norm', 'DWdivSW50', 'SW75', 'SW75_norm', 'DW75',
       'DW75_norm', 'SWaddDW75', 'SWaddDW75_norm', 'DWdivSW75', 'S1', 'S2',
       'S3', 'S4', 'AUCsys', 'AUCdia', 'S1_norm', 'S2_norm', 'S3_norm',
       'S4_norm', 'AUCsys_norm', 'AUCdia_norm', 'SQI_skew', 'SQI_kurtosis',
       'apg_a', 'apg_b', 'apg_c', 'apg_d', 'apg_e', 'ppg_a', 'ppg_b', 'ppg_c',
       'ppg_d', 'ppg_e', 'ratio_apg_b', 'ratio_apg_c', 'ratio_apg_d',
       'ratio_apg_e', 'ratio_ppg_b', 'ratio_ppg_c', 'ratio_ppg_d',
       'ratio_ppg_e', 'T_a', 'T_b', 'T_c

In [21]:
y

array([[1],
       [1],
       [1],
       ...,
       [2],
       [1],
       [1]], dtype=int64)

In [51]:
# fill nan to 0
X = X.fillna(0)

In [23]:
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# scaler = StandardScaler()
# # encodings_scaled = scaler.fit_transform(encodings)

# pca = PCA(n_components=97)

In [ ]:
scaler = StandardScaler()
encodings_scaled = scaler.fit_transform(X)

pca = PCA(n_components=20)
encodings_reduced = pca.fit_transform(encodings_scaled)

In [ ]:
encodings_reduced.shape

In [36]:
y = np.array(y).reshape(-1, 1)

In [57]:
encodings_reduced.max(),encodings_reduced.min(),encodings_reduced.mean(),encodings_reduced.std()

(490.59446872250106,
 -25.76702033256711,
 -3.605208436782023e-18,
 2.1940844031446582)

In [141]:
np.histogram(y,bins=5)

(array([1943,    0, 2617,    0, 2414], dtype=int64),
 array([0. , 0.4, 0.8, 1.2, 1.6, 2. ]))

In [56]:
encodings_reduced.shape

(87507, 20)

In [17]:
def LDDP_entropy(x, bins=100):
    """
    LDDPによる連続変数のエントロピー推定
    x: 1次元連続データ
    bins: 離散化ビンの数
    """
    hist, bin_edges = np.histogram(x, bins=bins, density=True)
    # 0の要素を除く
    hist = hist[hist > 0]
    H = -np.sum(hist * np.log(hist))  # 微分エントロピーに近似
    return H
print("bins",int(np.sqrt(len(y))))
H_y = LDDP_entropy(y, bins=int(np.sqrt(len(y))))
H_y

bins 300


5.657210370767604

In [14]:
features = pd.read_csv('F:\\minowa\\BloodPressureEstimation\\data\\results\\ppg_features_pulsedb_test_all.csv',index_col=0)

y = features['sbp'].values.reshape(-1,1)
X = features.drop(columns=["signal_index","subject","sbp","dbp"])
X = X.fillna(0)
scaler = StandardScaler()
scalery = StandardScaler()
encodings_scaled = scaler.fit_transform(X)
y_scaled = scalery.fit_transform(y)
pca = PCA(n_components=20)
encodings_reduced = pca.fit_transform(encodings_scaled)
emb_dim =20
co = ite.cost.MIShannon_DKL(
    mult=True, kl_co_name="BDKL_KnnK", kl_co_pars={"k": 100}
)
# co = ite.cost.MIShannon_HS(
#     mult=True, shannon_co_name="BHShannon_KnnK", shannon_co_pars={"k": 10}
# )
# stack = np.hstack((encodings_reduced, y_scaled))
# print(stack.shape)

val = co.estimation(np.hstack((encodings_reduced, y_scaled)), [emb_dim, 1])
print(f"MI value:{val}")
# emb_dim =20
# stack = np.hstack((encodings_reduced, y_scaled))
# print(stack.shape)
# val = co.estimation(stack, [emb_dim,1])
# print(f"MI value:{val}")

# Y のみでエントロピー推定
H_Y = LDDP_entropy(y_scaled, bins=int(np.sqrt(len(y))))
# co_HY = ite.cost.BHShannon_KnnK(mult=True, k=100)
# H_Y = co_HY.estimation(y_scaled.reshape(-1,1))
print(f"Entropy of target (H_Y): {H_Y}")
info_fraction = val / H_Y
print(f"Info-Fraction: {info_fraction}")

MI value:6.975802441620895


NameError: name 'LDDP_entropy' is not defined

In [18]:
H_Y = LDDP_entropy(y_scaled, bins=int(np.sqrt(len(y))))
# co_HY = ite.cost.BHShannon_KnnK(mult=True, k=100)
# H_Y = co_HY.estimation(y_scaled.reshape(-1,1))
print(f"Entropy of target (H_Y): {H_Y}")
info_fraction = val / H_Y
print(f"Info-Fraction: {info_fraction}")

Entropy of target (H_Y): 36.4072130650931
Info-Fraction: 0.19160495556605045


In [16]:
X.shape

(90289, 98)

In [15]:
X.columns

Index(['Tc', 'Ts', 'Td', 'Tsteepest', 'Steepest', 'TNegSteepest',
       'NegSteepest', 'TdiaRise', 'DiaRise', 'SteepDiaRise', 'TSystoDiaRise',
       'TdiaToEnd', 'Ratio', 'Ts_norm', 'Td_norm', 'Tsteepest_norm',
       'TNegSteepest_norm', 'TdiaRise_norm', 'TSystoDiaRise_norm',
       'TdiaToEnd_norm', 'SW25', 'SW25_norm', 'DW25', 'DW25_norm', 'SWaddDW25',
       'SWaddDW25_norm', 'DWdivSW25', 'SW50', 'SW50_norm', 'DW50', 'DW50_norm',
       'SWaddDW50', 'SWaddDW50_norm', 'DWdivSW50', 'SW75', 'SW75_norm', 'DW75',
       'DW75_norm', 'SWaddDW75', 'SWaddDW75_norm', 'DWdivSW75', 'S1', 'S2',
       'S3', 'S4', 'AUCsys', 'AUCdia', 'S1_norm', 'S2_norm', 'S3_norm',
       'S4_norm', 'AUCsys_norm', 'AUCdia_norm', 'SQI_skew', 'SQI_kurtosis',
       'apg_a', 'apg_b', 'apg_c', 'apg_d', 'apg_e', 'ppg_a', 'ppg_b', 'ppg_c',
       'ppg_d', 'ppg_e', 'ratio_apg_b', 'ratio_apg_c', 'ratio_apg_d',
       'ratio_apg_e', 'ratio_ppg_b', 'ratio_ppg_c', 'ratio_ppg_d',
       'ratio_ppg_e', 'T_a', 'T_b', 'T_c

In [26]:
y_scaled.shape

(90289, 1)

In [4]:
import normi

In [8]:
nmi = normi.NormalizedMI()
nmi.fit(np.hstack((encodings_reduced, y_scaled)))

NMI Estimation: 100%|██████████| 210/210 [00:28<00:00,  7.30it/s]


NormalizedMI()

In [11]:
nmi.mi_



array([[1.        , 0.24950743, 0.14418986, 0.23071748, 0.30415178,
        0.09181826, 0.15440525, 0.08853288, 0.10607724, 0.1059909 ,
        0.12598128, 0.05952238, 0.10711379, 0.14692663, 0.08795783,
        0.10686127, 0.05558863, 0.10695801, 0.08346583, 0.06594345,
        0.06384538],
       [0.24950743, 1.        , 0.06369214, 0.08578973, 0.13329494,
        0.14154514, 0.08904765, 0.0823105 , 0.18751399, 0.07603466,
        0.19314375, 0.05038646, 0.06580538, 0.14116976, 0.08531182,
        0.09553049, 0.07270253, 0.059911  , 0.05306916, 0.05775511,
        0.03015064],
       [0.14418986, 0.06369214, 1.        , 0.04585241, 0.21555431,
        0.26092038, 0.61210923, 0.04095515, 0.04191576, 0.0552125 ,
        0.0756094 , 0.07093676, 0.08569184, 0.04580937, 0.05835063,
        0.03108411, 0.06896625, 0.0627241 , 0.08297298, 0.06419235,
        0.05096265],
       [0.23071748, 0.08578973, 0.04585241, 1.        , 0.21807554,
        0.05132776, 0.06656374, 0.02884584, 0.0554411

In [15]:
import numpy as np
from scipy.spatial import KDTree
from scipy.special import digamma
from sklearn.preprocessing import StandardScaler
def mutual_information_ksg_full(X, y, k=5):
    if y.ndim == 1:
        y = y.reshape(-1, 1)
    n, dx = X.shape
    _, dy = y.shape
    xy = np.hstack((X, y))

    tree_xy = KDTree(xy)
    eps = tree_xy.query(xy, k=k + 1, p=np.inf)[0][:, -1]
    eps = np.nextafter(eps, 0)

    nx = KDTree(X).query_ball_point(X, r=eps, p=np.inf, return_length=True) - 1
    ny = KDTree(y).query_ball_point(y, r=eps, p=np.inf, return_length=True) - 1

    digamma_N = digamma(n)
    digamma_k = digamma(k)
    digamma_nx = np.mean(digamma(nx + 1))
    digamma_ny = np.mean(digamma(ny + 1))

    Ixy = max(digamma_N + digamma_k - digamma_nx - digamma_ny, 0)
    Hx = digamma_N - digamma_k + dx * np.mean(np.log(eps))
    Hy = digamma_N - digamma_k + dy * np.mean(np.log(eps))
    Hxy = digamma_N - digamma_k + (dx + dy) * np.mean(np.log(eps))

    return Ixy, Hx, Hy, Hxy


In [16]:
Ixy, Hx, Hy, Hxy = mutual_information_ksg_full(encodings_reduced, y_scaled, k=10)

In [18]:
Ixy, Hx, Hy, Hxy, Ixy / Hy

(0.5415387721710125,
 7.771189200019949,
 9.089621609519137,
 7.701798020572623,
 0.0595777025089674)

# cnn weight

In [1]:
class GradCAM:
    def __init__(self, model, feature_layer):
        self.model = model
        self.feature_layer = feature_layer
        self.model.eval()
        self.feature_grad = None
        self.feature_map = None
        self.hooks = []

        # 最終層逆伝播時の勾配を記録する
        # def save_feature_grad(module, in_grad, out_grad):
        #     self.feature_grad = out_grad[0]
        # self.hooks.append(self.feature_layer.register_backward_hook(save_feature_grad))

        # 最終層の出力 Feature Map を記録する
        def save_feature_map(module, inp, outp):
            self.feature_map = outp[0].detach()
        self.hooks.append(self.feature_layer.register_forward_hook(save_feature_map))

    def forward(self, x):
        return self.model(x)

    # def backward_on_target(self, output, target):
    #     self.model.zero_grad()
    #     one_hot_output = torch.zeros([1, output.size()[-1]],device=output.device)
    #     one_hot_output[0][target] = 1
    #     output.backward(gradient=one_hot_output, retain_graph=True)

    def clear_hook(self):
        for hook in self.hooks:
            hook.remove()

In [4]:
from resnet1d import ResNet1D
config = {
    "network":{
        "in_channels":1,
        "base_filters":64,
        "kernel_size":3,
        "stride":1,
        "groups":1,
        "n_block":5,
        "n_classes":2,
    },
    "learning_rate": 1e-3,
    "batch_size": 32,
    "epochs":100,
    "log_interval":100,
    "input_shape":[1,1250],
    "output_path": "..\\outputs\\resnet\\0228",
    "ppg":10    
          }
model = ResNet1D(**config["network"])

In [5]:
from sklearn.decomposition import PCA
pca = PCA(n_components=20)

In [6]:
ppg = np.load(r"F:\minowa\BloodPressureEstimation\data\processed\BP_npy\PulseDB\test_4.npy")
print(ppg.shape)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load('..\\outputs\\resnet\\0228\\best.pth'))
model.to(device)
model.eval()
loader = DataLoader(torch.from_numpy(ppg).float().to(device), batch_size=128, shuffle=False)
gradcam = GradCAM(model, model.basicblock_list[-1].conv2)
feature_maps = []
for X in tqdm(loader):
    # print(X.shape )
    output = gradcam.forward(X)
    # pred = torch.argmax(output, dim=1).item()
    feature_map = gradcam.feature_map.cpu().data.numpy()
    # print(feature_map.shape, feature_grad.shape) 
    # print(feature_map.shape)   
    feature_maps.extend(feature_map.reshape(128,-1))
feature_maps = np.stack(feature_maps)
print(feature_maps.shape)
pca = PCA(n_components=20)
feature_maps_pca = pca.fit_transform(feature_maps)

(111600, 1, 1250)


C:\Users\bsa\AppData\Local\Temp\ipykernel_22676\1254599787.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('..\\outputs\\resnet\\0228\\b

(111616, 1250)


In [7]:
feature_maps_pca = feature_maps_pca[:111600]
feature_maps_pca.shape

(111600, 20)

In [20]:
feature_maps_pca.shape

(111616, 20)

In [11]:
y = np.load(r"F:\minowa\BloodPressureEstimation\data\processed\BP_npy\PulseDB\test_sbp.npy").reshape(-1,1).squeeze()
y.shape

(111600,)

In [12]:
scaler = StandardScaler()
y_scaled = scaler.fit_transform(y.reshape(-1,1))

In [25]:
co = ite.cost.MIShannon_DKL(
            mult=True, kl_co_name="BDKL_KnnK", kl_co_pars={"k": 100}
        )
emb_dim = 20
print(feature_maps_pca.shape, y_scaled.shape)
val = co.estimation(np.hstack((feature_maps_pca, y_scaled)), [emb_dim, 1])
H_Y = LDDP_entropy(y_scaled, bins=int(np.sqrt(len(y_scaled))))
print(f"MI value:{val}")
print(f"Entropy of target (H_Y): {H_Y}")
info_fraction = val / H_Y
print(f"Info-Fraction: {info_fraction}")

(111600, 20) (111600, 1)
MI value:0.1728568700892172
Entropy of target (H_Y): 41.03853545032499
Info-Fraction: 0.0042120623504815725


In [13]:
import scipy
scipy.stats.entropy(np.histogram(y_scaled,bins=int(np.sqrt(len(y_scaled))))[0])

4.776519631958554

In [8]:
a = np.load("F:\\minowa\\BloodPressureEstimation\\repos\\PPG-BP-Analysis\\autoencoder\\autoencoder_results_emb20_nb100.npz")
a

In [23]:
# print keys
max_mi = a['mi'].max()
max_mi/ 41.03853545032499

0.008290494713720849